In [ ]:
# importing all the required libraries and providing the image dataset path
import numpy as np
import tensorflow as tf

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os


dataset_path = os.listdir('G:/Other computers/My_Laptop/Objective_1/Semantic Segmentation - U - Net/Efficient_Net_Classification/AI_Dataset/')

print (dataset_path)  #what kinds of classes are in this dataset

print("Types of classes labels found: ", len(dataset_path))

In [ ]:
# class labels for evaluating the loss function (categorical crossentropy)
class_labels = []

for item in dataset_path:
 # Get all the file names
 all_classes = os.listdir('G:/Other computers/My_Laptop/Objective_1/Semantic Segmentation - U - Net/Efficient_Net_Classification/AI_Dataset/' + '/' +item)
 #print(all_classes)

 # Add them to the list
 for room in all_classes:
    class_labels.append((item, str('dataset_path' + '/' +item) + '/' + room))
    #print(class_labels[:5])

In [ ]:
# Build a dataframe        
df = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])
print(df.head())
print(df.tail())

In [ ]:
# Let's check how many samples for each category are present
print("Total number of images in the dataset: ", len(df))

label_count = df['Labels'].value_counts()
print(label_count)

In [ ]:
#resizing all the input images to as per the pre-trained model requirement

import cv2
path = 'G:/Other computers/My_Laptop/Objective_1/Semantic Segmentation - U - Net/Efficient_Net_Classification/AI_Dataset/'
dataset_path = os.listdir('G:/Other computers/My_Laptop/Objective_1/Semantic Segmentation - U - Net/Efficient_Net_Classification/AI_Dataset/')

im_size = 300

images = []
labels = []

for i in dataset_path:
    data_path = path + str(i)  
    filenames = [i for i in os.listdir(data_path) ]
   
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

In [ ]:
#This model takes input images of shape (300, 300, 3), and the input data should range [0, 255]. 

images = np.array(images)

images = images.astype('float32') / 255.0
images.shape

In [ ]:
# encoding the labels to integers values for example chipping - 0, crater - 1, flank - 2
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y=df['Labels'].values
print(y)

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)

In [ ]:
y=y.reshape(-1,1)

from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y) #.toarray()
print(Y[:5])
print(Y[35:])

In [ ]:
#splitting the training and test dataset of the images
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, Y = shuffle(images, y, random_state=1)


train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.2, random_state=415)

#inspect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [ ]:
# labelling of the training data using one hot encoder
train_y = tf.keras.utils.to_categorical(train_y, 3)
test_y = tf.keras.utils.to_categorical(test_y, 3)

In [ ]:
train_y

In [ ]:
#importing the efficientnetB3 model 
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB3

NUM_CLASSES = 3
IMG_SIZE = 300
size = (IMG_SIZE, IMG_SIZE)


inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))


# Using model without transfer learning

outputs = EfficientNetB3(include_top=True, weights=None, classes=NUM_CLASSES)(inputs)

In [ ]:
#model compilation, optimiser selection and loss function selection
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"] )

model.summary()

hist = model.fit(train_x, train_y, epochs=30, verbose=2)

model.save('Efficient_Tool_Wear.hdf5')

In [ ]:
# importing libraries to plot the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

In [ ]:
class_names = [0,1,2]

In [ ]:
import numpy as np
rounded_labels=np.argmax(test_y, axis=1)
rounded_labels

In [ ]:
#class estimator for test image dataset
class estimator:
    _estimator_type = ''
    classes_=[]
    def __init__(self, model, classes):
        self.model = model
        self._estimator_type = 'classifier'
        self.classes_ = classes
    def predict(self, X):
        y_prob= self.model.predict(X)
        y_pred = y_prob.argmax(axis=1)
        return y_pred

classifier = estimator(model, class_names)

In [ ]:
#confusion matrix plot
plot_confusion_matrix(estimator=classifier, X=test_x, y_true=rounded_labels)

In [ ]:
#loss and test accuracy prediction as per the confusion matrix
preds = model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
#prediction code for new images which are not involved in the training

import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the trained model
model = tf.keras.models.load_model('Efficient_Tool_Wear_1.hdf5')

# Load and preprocess the image
img_path = 'G:/Other computers/My_Laptop/Objective_1/Clemson_Presentation/Testing_dataset/jpeg_images/2.jpg'
img = image.load_img(img_path, target_size=(300, 300))
img_array = image.img_to_array(img)

# Make a prediction
prediction = model.predict(img_array)

# Print the prediction
print(prediction)

In [ ]:
#prediction code for new images which are not involved in the training

from matplotlib.pyplot import imread
from matplotlib.pyplot import imshow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.applications.imagenet_utils import preprocess_input

img_path = 'G:/Other computers/My_Laptop/Objective_1/Clemson_Presentation/Testing_dataset/jpeg_images/16.jpg'


img = cv2.imread(img_path)
img = cv2.resize(img, (300, 300))

x = np.expand_dims(img, axis=0)
x = preprocess_input(x)

print('Input image shape:', x.shape)

my_image = imread(img_path)
imshow(img)



preds=model.predict(x)
print("predicted class: ", preds )    # probabilities for being in each of the 3 classes

In [ ]:
#prediction code for new images which are not involved in the training

from matplotlib.pyplot import imread
from matplotlib.pyplot import imshow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.applications.imagenet_utils import preprocess_input

Img_path = "G:/Other computers/My_Laptop/Objective_1/Clemson_Presentation/Testing_dataset/jpeg_images/16.jpg"


img = tf.keras.utils.load_img(
    Img_path, target_size=(300, 300)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])


print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)